In [3]:
import os
import shutil

def rename_and_move_wav_files(main_folder, new_folder):
    # Create the new folder if it doesn't exist
    os.makedirs(new_folder, exist_ok=True)

    for root, dirs, files in os.walk(main_folder):
        for dir in dirs:
            sub_folder = os.path.join(root, dir)
            folder_name = os.path.basename(sub_folder)
            file_names = os.listdir(sub_folder)

            for index, file_name in enumerate(file_names):
                file_path = os.path.join(sub_folder, file_name)

                if file_name.lower().endswith(".wav"):
                    new_file_name = f"{folder_name}_{index}.wav"
                    new_file_path = os.path.join(new_folder, new_file_name)
                    shutil.copy2(file_path, new_file_path)
                    print(f"Copied: {file_name}")

# Provide the main folder path and the new folder path
main_folder = "/Users/dilipharish/Master-Thesis/dh-freesound-crossmodal-search/data/audio/dh-evaluation"
new_folder = "/Users/dilipharish/Master-Thesis/dh-freesound-crossmodal-search/data/audio/dh-evaluation-renamed"

# Call the rename_and_move_wav_files function
rename_and_move_wav_files(main_folder, new_folder)


Copied: soundscape_train_uniform4.wav
Copied: soundscape_train_uniform3.wav
Copied: soundscape_train_uniform2.wav
Copied: soundscape_train_uniform0.wav
Copied: soundscape_train_uniform1.wav
Copied: soundscape_train_uniform4.wav
Copied: soundscape_train_uniform3.wav
Copied: soundscape_train_uniform2.wav
Copied: soundscape_train_uniform0.wav
Copied: soundscape_train_uniform1.wav
Copied: soundscape_train_uniform4.wav
Copied: soundscape_train_uniform3.wav
Copied: soundscape_train_uniform2.wav
Copied: soundscape_train_uniform0.wav
Copied: soundscape_train_uniform1.wav
Copied: soundscape_train_uniform4.wav
Copied: soundscape_train_uniform3.wav
Copied: soundscape_train_uniform2.wav
Copied: soundscape_train_uniform0.wav
Copied: soundscape_train_uniform1.wav
Copied: soundscape_train_uniform4.wav
Copied: soundscape_train_uniform3.wav
Copied: soundscape_train_uniform2.wav
Copied: soundscape_train_uniform0.wav
Copied: soundscape_train_uniform1.wav
Copied: soundscape_train_uniform4.wav
Copied: soun